Table 4.8.1: Source of referral into treatment, new presentations to treatment

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['Table 4.8.1']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
cell = tab.filter('Referral Source')
cell.assert_one()
obs = tab.filter('Inconsistent/missing').shift(0,1).expand(RIGHT) | tab.filter('Other YP').shift(0,-1).expand(RIGHT)
observations = tab.filter('n').fill(DOWN).is_not_blank().is_not_whitespace() - obs
ReferralSource = cell.expand(DOWN).is_not_blank().is_not_whitespace() | \
                  cell.shift(1,0).expand(DOWN).is_not_blank().is_not_whitespace()
Clients = cell.expand(RIGHT).is_not_blank().is_not_whitespace()
Dimensions = [
            HDim(ReferralSource,'Treatment group',DIRECTLY,LEFT),
            HDim(Clients,'Clients in treatment',CLOSEST,LEFT),
            HDimConst('Measure Type','Count'),
            HDimConst('Unit','new-presentations-to-treatment'),
            HDimConst('Period', 'gregorian-interval/2017-04-01T00:00:00/P1Y')
            ]

In [4]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [5]:
new_table = c1.topandas()

In [6]:
import numpy as np
new_table['OBS'].replace('', np.nan, inplace=True)
new_table.dropna(subset=['OBS'], inplace=True)
new_table.rename(columns={'OBS': 'Value'}, inplace=True)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Value'] = new_table['Value'].map(lambda x:''
                                  if (x == '-')
                                  else int(x))

In [7]:
new_table['Treatment group'].unique()

array(['Self', 'Other family and friends',
       'Self, family and friends subtotal', 'GP', 'Hospital',
       'Social services', 'Health – other', 'Other community health',
       'Psychiatry', 'A&E', 'Syringe Exchange',
       'Community care assessment', 'Children social services',
       'Health services and social care subtotal', 'Arrest referral/DIP',
       'Prison', 'Probation', 'Criminal justice – other', 'ATR', 'DRR',
       'Community rehabilitation company', 'Liaison and diversion',
       'Other criminal justice', 'Criminal justice subtotal',
       'Drug service statutory', 'Drug service non-statutory',
       'Community alcohol team', 'Substance misuse service subtotal',
       'Other', 'Other YP', 'Job centre plus', 'Employment service',
       'Other sex worker project', 'Other treatment provider',
       'Connexions', 'Education service', 'LAC', 'Employer',
       'Other helplines & websites', 'Other subtotal', 'Total',
       'Inconsistent/missing'], dtype=object)

In [8]:
new_table['Basis of treatment'] = new_table['Treatment group'].map(
    lambda x: {
        'Referral Source':'referral-source',
        'Self, family and friends subtotal':'self-family-and-friends-subtotal',
        'Self':'self-family-and-friends-subtotal/self',
        'Other family and friends':'self-family-and-friends-subtotal/other-family-and-friends',
        'Health services and social care subtotal':'health-services-and-social-care-subtotal',
        'GP':'health-services-and-social-care-subtotal/gp',
        'Hospital':'health-services-and-social-care-subtotal/hospital',
        'Social services':'health-services-and-social-care-subtotal/social-services',
        'Health – other':'health-services-and-social-care-subtotal/health-other',
        'Other community health':'health-services-and-social-care-subtotal/health-other/other-community-health',
        'Psychiatry':'health-services-and-social-care-subtotal/health-other/psychiatry',
        'A&E':'health-services-and-social-care-subtotal/health-other/a-and-e',
        'Syringe Exchange':'health-services-and-social-care-subtotal/health-other/syringe-exchange',
        'Community care assessment':'health-services-and-social-care-subtotal/health-other/community-care-assessment',
        'Children social services':'health-services-and-social-care-subtotal/health-other/children-social-services',
        'Criminal justice subtotal':'criminal-justice-subtotal',
        'Arrest referral/DIP':'criminal-justice-subtotal/arrest-referral/dip',
        'Prison':'criminal-justice-subtotal/prison',
        'Probation':'criminal-justice-subtotal/probation',
        'Criminal justice – other':'criminal-justice-subtotal/criminal-justice-other',
        'ATR':'criminal-justice-subtotal/criminal-justice-other/atr',
        'DRR':'criminal-justice-subtotal/criminal-justice-other/drr',
        'Community rehabilitation company':'criminal-justice-subtotal/criminal-justice-other/community-rehabilitation-company',
        'Liaison and diversion':'criminal-justice-subtotal/criminal-justice-other/liaison-and-diversion',
        'Other criminal justice':'criminal-justice-subtotal/criminal-justice-other/other-criminal-justice',
        'Substance misuse service subtotal':'substance-misuse-service-subtotal',
        'Drug service statutory':'substance-misuse-service-subtotal/drug-service-statutory',
        'Drug service non-statutory':'substance-misuse-service-subtotal/drug-service-non-statutory',
        'Community alcohol team':'substance-misuse-service-subtotal/community-alcohol-team',
        'Other subtotal':'other-subtotal',
        'Other':'other-subtotal/other',
        'Other':'other-subtotal/other/other',
        'Other YP':'other-subtotal/other/other-yp',
        'Job centre plus':'other-subtotal/other/job-centre-plus',
        'Employment service':'other-subtotal/other/employment-service',
        'Other sex worker project':'other-subtotal/other/other-sex-worker-project',
        'Other treatment provider':'other-subtotal/other/other-treatment-provider',
        'Connexions':'other-subtotal/other/connexions',
        'Education service':'other-subtotal/other/education-service',
        'LAC':'other-subtotal/other/lac',
        'Employer':'other-subtotal/other/employer',
        'Other helplines & websites':'other-subtotal/other/other-helplines-and-websites',
        'Inconsistent/missing':'inconsistent/missing',
        'Total':'total'        
        }.get(x, x))

In [9]:
import urllib.request as request
import csv
import io
import requests
vrl="https://raw.githubusercontent.com/ONS-OpenData/ref_alcohol/master/codelists/substance-type.csv"
t=requests.get(vrl).content
g=pd.read_csv(io.StringIO(t.decode('utf-8')))
new_table = pd.merge(new_table, g, how = 'left', left_on = 'Clients in treatment', right_on = 'Label')
new_table.columns = ['Substance type' if x=='Notation' else x for x in new_table.columns]
new_table = new_table[['Period','Basis of treatment','Substance type','Measure Type','Value','Unit']]

In [10]:
new_table

,Period,Basis of treatment,Substance type,Measure Type,Value,Unit
0,gregorian-interval/2017-04-01T00:00:00/P1Y,self-family-and-friends-subtotal/self,opiate,Count,21542,new-presentations-to-treatment
1,gregorian-interval/2017-04-01T00:00:00/P1Y,self-family-and-friends-subtotal/self,non-opiate-only,Count,9747,new-presentations-to-treatment
2,gregorian-interval/2017-04-01T00:00:00/P1Y,self-family-and-friends-subtotal/self,non-opiate-and-alcohol,Count,10891,new-presentations-to-treatment
3,gregorian-interval/2017-04-01T00:00:00/P1Y,self-family-and-friends-subtotal/self,alcohol-only,Count,29842,new-presentations-to-treatment
4,gregorian-interval/2017-04-01T00:00:00/P1Y,self-family-and-friends-subtotal/self,total,Count,72022,new-presentations-to-treatment
5,gregorian-interval/2017-04-01T00:00:00/P1Y,self-family-and-friends-subtotal/other-family-...,opiate,Count,450,new-presentations-to-treatment
6,gregorian-interval/2017-04-01T00:00:00/P1Y,self-family-and-friends-subtotal/other-family-...,non-opiate-only,Count,364,new-presentations-to-treatment
7,gregorian-interval/2017-04-01T00:00:00/P1Y,self-family-and-friends-subtotal/other-family-...,non-opiate-and-alcohol,Count,327,new-presentations-to-treatment
8,gregorian-interval/2017-04-01T00:00:00/P1Y,self-family-and-friends-subtotal/other-family-...,alcohol-only,Count,700,new-presentations-to-treatment
9,gregorian-interval/2017-04-01T00:00:00/P1Y,self-family-and-friends-subtotal/other-family-...,total,Count,1841,new-presentations-to-treatment


In [11]:
new_table['Basis of treatment'].unique()

array(['self-family-and-friends-subtotal/self',
       'self-family-and-friends-subtotal/other-family-and-friends',
       'self-family-and-friends-subtotal',
       'health-services-and-social-care-subtotal/gp',
       'health-services-and-social-care-subtotal/hospital',
       'health-services-and-social-care-subtotal/social-services',
       'health-services-and-social-care-subtotal/health-other',
       'health-services-and-social-care-subtotal/health-other/other-community-health',
       'health-services-and-social-care-subtotal/health-other/psychiatry',
       'health-services-and-social-care-subtotal/health-other/a-and-e',
       'health-services-and-social-care-subtotal/health-other/syringe-exchange',
       'health-services-and-social-care-subtotal/health-other/community-care-assessment',
       'health-services-and-social-care-subtotal/health-other/children-social-services',
       'health-services-and-social-care-subtotal',
       'criminal-justice-subtotal/arrest-referral/d